In [274]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [275]:
## loading the dataset
data =pd.read_csv("Churn_Modelling.csv")
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [276]:
# dropiing unesseccary col
data =data.drop(columns=["RowNumber", "CustomerId", "Surname"], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [277]:
# encoding gender col
label =LabelEncoder()
data["Gender"] =label.fit_transform(data["Gender"])

In [278]:
geo =np.array(data["Geography"]).reshape(len(np.array(data["Geography"])), 1)

In [279]:
ohe =OneHotEncoder()
ohe_cols =ohe.fit_transform(geo)
ohe_cols

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [280]:
df =pd.DataFrame(ohe_cols.toarray(), columns= ohe.get_feature_names_out())
df

,x0_France,x0_Germany,x0_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [281]:
len(df)==len(data)


True

In [282]:
data.drop(columns=["Geography"], inplace=True)
data =pd.concat([data, df], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,x0_France,x0_Germany,x0_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [283]:
## saveing the state of encode decoder
with open("labelencoder_gender.pkl", "wb") as f:
    pickle.dump(label, f)

with open("onehotencoder_geography.pkl", "wb") as f:
    pickle.dump(ohe, f)

In [284]:
# divide the independent and dependent dataset
X =data.drop("Exited", axis=1)
Y =data["Exited"]

In [285]:
# test train spit
X_train, X_test, Y_train, Y_test =train_test_split(X, Y, test_size=0.2, random_state=42)
# standard scale
standardscaler =StandardScaler()
X_train =standardscaler.fit_transform(X_train)
X_test =standardscaler.transform(X_test)

In [286]:
with open("standardscaler.pkl", "wb") as f:
    pickle.dump(standardscaler, f)

### ANN implementation

In [287]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import datetime

In [288]:
## Building ANN Model

model =Sequential([
    Dense(64, activation="relu", input_shape =(X_train.shape[1],)), #hl1 connected with input layer
    Dense(32, activation="relu"), # hl2
    Dense(1, activation="sigmoid") #outputlayer
])

c:\Users\faiqi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [289]:
## getting summary of model
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [290]:
## compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [209]:
## setup the tensorboard
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
dir ="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback =TensorBoard(log_dir =dir, histogram_freq=1)

In [210]:
earlystop_callback =EarlyStopping(monitor="val-lose", patience=10,mode ="min", restore_best_weights=True)

In [291]:
## Trainng the model
history =model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100)

Epoch 1/100


250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7695 - loss: 0.5003 - val_accuracy: 0.8355 - val_loss: 0.3894
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8357 - loss: 0.3999 - val_accuracy: 0.8490 - val_loss: 0.3549
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8508 - loss: 0.3664 - val_accuracy: 0.8590 - val_loss: 0.3466
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8672 - loss: 0.3299 - val_accuracy: 0.8600 - val_loss: 0.3390
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8633 - loss: 0.3264 - val_accuracy: 0.8575 - val_loss: 0.3432
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8632 - loss: 0.3312 - val_accuracy: 0.8615 - val_loss: 0.3370
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8632 - loss: 0.3288 - val_accuracy: 0.8585 - val_loss: 0.3387
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8676 - loss: 0.3261 - val_accuracy: 0.8640

In [293]:
model.save("model.h5")

In [294]:
from tensorflow.keras.models import load_model

In [295]:
### Making prediction
model =load_model("model.h5")
#loading scaler 
with open("standardscaler.pkl", "rb") as f:
    standard =pickle.load(f)
with open("labelencoder_gender.pkl", "rb") as f:
    labelencoder =pickle.load(f)
with open("onehotencoder_geography.pkl", "rb") as f:
    ohe = pickle.load(f)

In [318]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,x0_France,x0_Germany,x0_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [319]:
# Example input data
inputdata ={
    "CreditScore": 619,
    "Geography": "France",
    "Gender": "Female",
    "Age": 42,
    "Tenure": 2,
    "Balance": 0.00,
    "NumOfProducts": 1,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 101348.88


}

In [320]:
geo =ohe.transform(np.array(inputdata["Geography"]).reshape(1,1)).toarray()
geo

array([[1., 0., 0.]])

In [321]:
geo =pd.DataFrame(geo, columns=ohe.get_feature_names_out())
geo

,x0_France,x0_Germany,x0_Spain
0,1.0,0.0,0.0


In [322]:
gen =label.transform(np.array(inputdata["Gender"]).reshape(1,1))
gen

c:\Users\faiqi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([0])

In [323]:
gen =pd.DataFrame(gen , columns=["Gender"])
gen

,Gender
0,0


In [324]:
input_df =pd.DataFrame({
    "CreditScore": [619],
    "Geography": ["France"],
    "Gender": ["Female"],
    "Age": [42],
    "Tenure": [2],
    "Balance": [0.00],
    "NumOfProducts": [1],
    "HasCrCard": [1],
    "IsActiveMember": [1],
    "EstimatedSalary": [101348.88]


})
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.0,1,1,1,101348.88


In [325]:
input_df.drop(columns =["Geography", "Gender"], inplace =True)

In [326]:
input_df =pd.concat([input_df, geo, gen], axis =1)
input_df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,x0_France,x0_Germany,x0_Spain,Gender
0,619,42,2,0.0,1,1,1,101348.88,1.0,0.0,0.0,0


In [328]:

input_df =standard.transform(np.array(input_df))
input_df =pd.concat([pd.DataFrame(input_df[:,0]), pd.DataFrame(input_df[:,-1]), pd.DataFrame(input_df[:,1:-1])], axis =1)

c:\Users\faiqi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [329]:
input_df

,0,0,0,1,2,3,4,5,6,7,8,9
0,-0.338808,-0.576388,83.251125,-3.507959,-1.736467,-1.218455,-0.916688,0.649203,202762.668433,-1.746148,-0.998501,-0.579467


In [330]:
model.predict(input_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


array([[1.]], dtype=float32)